# Multi-touch Force Exploration

This notebook provides quick exploratory analysis utilities for inspecting the force (`fz`) distribution across all CSV files in a directory.


In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 30)


In [2]:
def summarize_fz(directory: str, threshold: float = 1.0, pattern: str = "*.csv") -> pd.DataFrame:
    """Summarize how many rows fall below the absolute `fz` threshold for every CSV in `directory`.

    Parameters
    ----------
    directory: str
        Path to the folder containing CSV files.
    threshold: float
        Absolute-value cutoff applied to `fz`.
    pattern: str
        Glob pattern for CSV selection (default: '*.csv').
    """
    base_path = Path(directory)
    if not base_path.exists():
        raise FileNotFoundError(f"Directory not found: {directory}")

    rows = []
    for csv_path in sorted(base_path.glob(pattern)):
        try:
            df = pd.read_csv(csv_path)
        except Exception as exc:
            rows.append({
                "file": csv_path.name,
                "total_rows": np.nan,
                "valid_fz_rows": np.nan,
                "abs_fz_lt_threshold": np.nan,
                "abs_fz_ge_threshold": np.nan,
                "note": f"Failed to read: {exc}"
            })
            continue

        if "fz" not in df.columns:
            rows.append({
                "file": csv_path.name,
                "total_rows": len(df),
                "valid_fz_rows": 0,
                "abs_fz_lt_threshold": np.nan,
                "abs_fz_ge_threshold": np.nan,
                "note": "Missing 'fz' column"
            })
            continue

        fz = pd.to_numeric(df["fz"], errors="coerce")
        valid_mask = ~fz.isna()
        valid_count = valid_mask.sum()
        abs_fz = fz[valid_mask].abs()

        below = (abs_fz < threshold).sum()
        above = (abs_fz >= threshold).sum()

        rows.append({
            "file": csv_path.name,
            "total_rows": len(df),
            "valid_fz_rows": valid_count,
            "abs_fz_lt_threshold": below,
            "abs_fz_ge_threshold": above,
            "pct_fz_lt_threshold": below / valid_count * 100 if valid_count else np.nan,
            "note": ""
        })

    summary = pd.DataFrame(rows)
    if not summary.empty:
        summary = summary.sort_values("file").reset_index(drop=True)
    return summary


In [ ]:
multi_touch_dir = "../Database/Data/multi_touch_data"
summary = summarize_fz(multi_touch_dir, threshold=1.0, pattern="*.csv")
summary


,file,total_rows,valid_fz_rows,abs_fz_lt_threshold,abs_fz_ge_threshold,pct_fz_lt_threshold,note
0,multi_touch_1_2_20251109_230349.csv,589,589,271,318,46.010187,
1,multi_touch_1_5_20251109_232135.csv,415,415,82,333,19.759036,
2,multi_touch_1_6_20251109_234938.csv,360,360,46,314,12.777778,
3,multi_touch_1_7_20251109_233225.csv,409,409,83,326,20.293399,
4,multi_touch_1_9_20251109_235456.csv,406,406,152,254,37.438424,
5,multi_touch_2_3_20251109_230603.csv,607,607,204,403,33.607908,
6,multi_touch_2_7_20251109_234435.csv,382,382,56,326,14.659686,
7,multi_touch_2_9_20251109_234738.csv,346,346,65,281,18.786127,
8,multi_touch_3_5_20251109_232304.csv,334,334,29,305,8.682635,
9,multi_touch_3_7_20251109_235659.csv,449,449,100,349,22.271715,


In [4]:
totals = summary[["valid_fz_rows", "abs_fz_lt_threshold", "abs_fz_ge_threshold"]].sum()
shares = {
    "pct_lt_threshold": totals["abs_fz_lt_threshold"] / totals["valid_fz_rows"] * 100 if totals["valid_fz_rows"] else np.nan,
    "pct_ge_threshold": totals["abs_fz_ge_threshold"] / totals["valid_fz_rows"] * 100 if totals["valid_fz_rows"] else np.nan,
}

totals.to_frame(name="count"), shares


(                     count
 valid_fz_rows         8572
 abs_fz_lt_threshold   2099
 abs_fz_ge_threshold   6473,
 {'pct_lt_threshold': np.float64(24.486700886607558),
  'pct_ge_threshold': np.float64(75.51329911339243)})